## Google Colab dep installation

In [ ]:
!git clone "https://github.com/PredelinaAsya/ModelCompression.git"
%cd ModelCompression/hw3
!pip3 install -qr "requirements.txt"

## Importing deps

In [175]:
import numpy as np
import pickle
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
from tqdm import tqdm
from ultralytics import YOLO

## Clustering

In [176]:
def apply_weight_clustering(layer, max_clusters=64, collect_data=False):
    original_shape = layer.weight.data.shape
    flattened_weights = layer.weight.data.cpu().numpy().flatten().reshape(-1, 1)
    unique_weights = np.unique(flattened_weights)
    n_clusters = min(len(unique_weights), max_clusters)
    dynamic_n_init = min(10, n_clusters // 2)

    centroids, indices = None, None

    if n_clusters > 1:
        kmeans = KMeans(n_clusters=n_clusters, n_init=dynamic_n_init, random_state=0).fit(flattened_weights)
        new_weights = np.array([kmeans.cluster_centers_[label] for label in kmeans.labels_])
        new_weights = new_weights.reshape(original_shape)
        layer.weight.data = torch.Tensor(new_weights).to(layer.weight.device)

        if collect_data:
            centroids = kmeans.cluster_centers_
            indices = kmeans.labels_

    return centroids, indices

Loading the model

In [177]:
model = YOLO("yolov8n.pt")

Verifying of model performance before clustering

In [167]:
metrics = model.val(data="coco128.yaml")

Ultralytics YOLOv8.0.190 🚀 Python-3.10.9 torch-2.0.1 CPU (Apple M1 Pro)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /Users/danielkopecky/DataspellProjects/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:20<00:00,  2.56s/it]
                   all        128        929       0.64      0.537      0.605      0.446
                person        128        254      0.797      0.677      0.764      0.538
               bicycle        128          6      0.514      0.333      0.315      0.264
                   car        128         46      0.813      0.217      0.273      0.168
            motorcycle        128          5      0.687      0.887      0.898      0.685
              airplane        128          6       0.82      0.833      0.927      0.675
 

Applying clustering weights to each convolutional layer

In [178]:
centroid_dict = {}
index_dict = {}

total = sum(1 for layer in model.modules() if isinstance(layer, nn.Conv2d))
with tqdm(total=total, desc="Processing layers", dynamic_ncols=True) as pbar:
    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d):
            centroids, indices = apply_weight_clustering(layer, collect_data=True)
            if centroids is not None and indices is not None:
                centroid_dict[name] = centroids.tolist()
                index_dict[name] = indices.tolist()
            pbar.update(1)

Processing layers: 100%|██████████| 64/64 [01:12<00:00,  1.14s/it]


Saving centroids and indices

In [179]:
with open('centroids.pkl', 'wb') as f:
    pickle.dump(centroid_dict, f)

with open('indices.pkl', 'wb') as f:
    pickle.dump(index_dict, f)

Verifying of model performance after clustering

In [180]:
metrics = model.val(data="coco128.yaml")

Ultralytics YOLOv8.0.190 🚀 Python-3.10.9 torch-2.0.1 CPU (Apple M1 Pro)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /Users/danielkopecky/DataspellProjects/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:20<00:00,  2.61s/it]
                   all        128        929      0.588      0.532      0.589      0.437
                person        128        254        0.7      0.673      0.753      0.529
               bicycle        128          6      0.532      0.333      0.304      0.219
                   car        128         46      0.715      0.217      0.264       0.17
            motorcycle        128          5      0.671      0.819      0.938       0.72
              airplane        128          6      0.672      0.667      0.893      0.678
 